# Finding ACMG Genes

Selected variants will be annotated with the ACMG actionable gene table, Kaviar frequency, ClinVar clinical significance ratings, DANN and CADD predicted pathogenicity scores for SNV's, and coding consequence predictions using SnpEff (table in progress).

Variants will be filtered for appropriate mode of inheritance and then labeled as predictive when there is presence of mutation(s) with appropriate inheritance (eg, 2 bi-allelic pathogenic mutations for a recessive disorder).  

Mutations defined strictly as either:
- Annotated in ClinVar with a clinical significance of 4 or 5, but never 2 or 3 or labeled pathogenic in HGMD (to be added later)  
OR    
- Novel in Kaviar with user-specified frequency and predicted to be disease-causing with a SnpEff impact score of 'high' 

## User Specified Parameters

In [122]:
# specify your impala host
impala_host = 'ec2-54-86-98-154.compute-1.amazonaws.com'

# enter maximum kaviar frequency to return
kav_freq = '.03'

# enter sample id's in list format or 'all'
sample_list = ['102-00317-01', '102-00317-02', '102-00317-03a']
#sample_list = 'all'
# TODO: add ability to enter only family id and parition tables on family id

# enter yes to retain only variants with "HIGH" impact coding consequences
coding_cons = 'no'
# functionality to be added when snpeff table complete

########################
## database locations ##
########################
# enter database.name for variant table
var_table = 'training.illumina_vars'
# enter database.name of global variants table
gv_table = 'training.global_vars'
# enter database.name of acmg_ensembl table
acmg_table = 'p7_ref_grch37.acmg_ensembl'
# enter user database to output tables
out_db = 'training'
# enter desired basename for output files
out_name = 'acmg_genes'

## Adding positional information to the ACMG gene list¶

Chromosome, start and stop positions were added to the ACMG Gene list by joining it with the ensembl_genes table using gene name.

    CREATE TABLE acmg_ensembl AS (
        SELECT acmg.*, ens.chrom, ens.start, ens.stop, ens.gene_name, 
        ens.gene_id, ens.transcript_id
        FROM p7_ref_grch37.acmg_genes acmg, p7_ref_grch37.ensembl_genes ens
        WHERE acmg.gene = ens.gene_name
     )

The results were saved as training.acmg_ensembl

## Parse User Arguments

In [123]:
# format sample id argument
sample_arg = []
if sample_list != 'all' and len(sample_list) > 1:
    sample_arg.append("AND ill.subject_id IN ('" + "' , '".join(sample_list) + "')")
    subject_statement = ", ".join(str(i) for i in sample_arg)
else: 
    print "This analysis is designed for trios only."

In [124]:
# view query parameters
print subject_statement

AND ill.subject_id IN ('102-00317-01' , '102-00317-02' , '102-00317-03a')


## Connecting to Impala 

In [125]:
# connect to impala
from impala.dbapi import connect
from impala.util import as_pandas

# to connect to specific database, use the database argument
conn=connect(host=impala_host, port=21050)

In [126]:
acmg_query = '''
-- find pathogenic illumina variants in acmg regions
with vars as (
select ill.*, acmg.phenotype, acmg.mim_disorder, acmg.pmid_entry, acmg.age_onset,
    acmg.mim_gene, acmg.inheritance, acmg.gene_id, acmg.transcript_id
from {} as ill, {} acmg
WHERE ill.chrom = acmg.chrom
AND ill.pos BETWEEN acmg.start and acmg.stop
{}
)
  
-- annotate and filter illumina variants
select distinct vars.*, gv.rs_id, gv.strand, gv.gene_name, gv.gene_id, gv.transcript_name, gv.transcript_id,
    gv.clin_sig, gv.clin_dbn, gv.kav_freq, gv.kav_source, gv.dbsnp_build, gv.var_type, gv.cadd_raw,
    gv.dann_score, gv.interpro_domain
from vars, {} as gv
where vars.transcript_id = gv.transcript_id
-- subset for rare variants with high impact or clinvar significant
and ((gv.kav_freq < {} or gv.kav_freq is null) 
    or 
    (gv.clin_sig NOT REGEXP '3|2[^5]|2$'  
                     AND  gv.clin_sig REGEXP '4|[^25]5|^5'))
'''.format(var_table, acmg_table, subject_statement, gv_table, kav_freq)

# open connection, run query, close connection 
def run_query(query_name):
    cur = conn.cursor()
    # run query 
    print 'Running the following query on impala: \n' + query_name
    cur = conn.cursor()
    cur.execute(query_name)
    df = as_pandas(cur)
    cur.close()
    return df

# run kaviar annotation query
acmg_df = run_query(acmg_query)

Running the following query on impala: 

-- find pathogenic illumina variants in acmg regions
with vars as (
select ill.*, acmg.phenotype, acmg.mim_disorder, acmg.pmid_entry, acmg.age_onset,
    acmg.mim_gene, acmg.inheritance, acmg.gene_id, acmg.transcript_id
from training.illumina_vars as ill, p7_ref_grch37.acmg_ensembl acmg
WHERE ill.chrom = acmg.chrom
AND ill.pos BETWEEN acmg.start and acmg.stop
AND ill.subject_id IN ('102-00317-01' , '102-00317-02' , '102-00317-03a')
)
  
-- annotate and filter illumina variants
select distinct vars.*, gv.rs_id, gv.strand, gv.gene_name, gv.gene_id, gv.transcript_name, gv.transcript_id,
    gv.clin_sig, gv.clin_dbn, gv.kav_freq, gv.kav_source, gv.dbsnp_build, gv.var_type, gv.cadd_raw,
    gv.dann_score, gv.interpro_domain
from vars, training.global_vars as gv
where vars.transcript_id = gv.transcript_id
-- subset for rare variants with high impact or clinvar significant
and ((gv.kav_freq < .03 or gv.kav_freq is null) 
    or 
    (gv.clin_sig NOT RE

In [127]:
# verify results were found before continuning
if len(acmg_df) > 0:
    print str(len(acmg_df)) + ' variants located in ACMG regions.'
else: 
    print "No variants located in ACMG regions."

No variants located in ACMG regions.


## Examine MOI

In [ ]:
# subset data frame by trio member
newborns = acmg_df[acmg_df['subject_id'].endswith('03')]
mothers = acmg_df[acmg_df['subject_id'].endswith('01')]
fathers = acmg_df[acmg_df['subject_id'].endswith('02')]

The newborn subset was split to report predictive variants for:  

- All variants in regions of dominant disorders  
- All homozygous recessive variants in regions of autosomal recessive disorders  
- All heterozygous variants in autosomal recessive regions for downstream analysis of compound heterozygosity

In [ ]:
# disable erroneous pandas warning
pd.options.mode.chained_assignment = None

# subset variants by variant MOI and/or zygosity
nb_dominant = newborns[((newborns['inheritance'] == 'AD') & (newborns['predictive'] == True))]
nb_dominant.name = 'dominant'

nb_hom_recessive = newborns[((newborns['inheritance'] == 'AR') & (newborns['gt'] == '1/1') \
                             & (newborns['predictive'] == True))]
nb_hom_recessive.name = 'hom_recessive'

nb_het = newborns[((newborns['inheritance'] == 'AR') & (newborns['gt'] == '0/1') \
                   & (newborns['predictive'] == True))]

### Locate Potential Compound Heterozygots

Newborn het predictive variants were subset for variants inherited from het parents, and then grouped by gene and family:

In [ ]:
# subset het parent variants, as homozygous parents are not of interest
mom_het = mothers.loc[mothers['gt'] == '0/1']
dad_het = fathers.loc[fathers['gt'] == '0/1']
           
# function to find matching parent variants
def find_parent_vars(nb_df, parent_df):
    # merge dataframes on variant id
    merged_df = pd.merge(nb_df, parent_df, on=['var_id'], how='inner')
    # rename parent sample_id column to avoid dropping when removing '_y' cols
    merged_df.rename(columns = {'member_y':'from_parent'}, inplace=True)
    # drop extra y columns from merge with fathers
    drop_y(merged_df)
    #remove _x from colnames
    merged_df.rename(columns=lambda x: x.replace('_x', ''), inplace=True)
    return merged_df
    
# run function for each parent set
if (len(mom_het) > 0) and (len(dad_het) > 0):
    nb_and_mom = find_parent_vars(nb_het, mom_het)
    nb_and_dad = find_parent_vars(nb_het, dad_het)
    # merge variants found in either mother or father
    het_cands = pd.concat([nb_and_mom,nb_and_dad]).drop_duplicates().reset_index(drop=True)
    # group variants by gene name
    by_gene = het_cands.groupby(['gene', 'family_id'])
else:
    print "No compound het variants"

After grouping the variants by gene and family, the variants will be filtered to keep only variants with at least one different variant coming from the mother and one from the father.

In [ ]:
# function to find compound hets
def find_comphets(gene_group, comphet_list_name):
    for name, group in gene_group:
        # if there is a variant in more than one position
        if group.pos.nunique() > 1:
            # and there are more than one variants from both parents
            if len(group[group['from_parent'] == 'M'] > 1) and len(group[group['from_parent'] == 'F'] > 1):
                comphet_list_name.append(group)
            # or if there is only one variant from each parent
            elif len(group[group['from_parent'] == 'M'] == 1) and len(group[group['from_parent'] == 'F'] == 1):
                # and those variants are different
                if len(group[group['from_parent'] == 'M'].pos - group[group['from_parent'] == 'F']) > 0:
                        comphet_list_name.append(group)

# create empty list to store comp_hets
comp_hets = []

if len(by_gene) > 0:
    # run function on by_gene
    find_comphets(by_gene, comp_hets)
    # combine results into dataframe
    comphet_df = pd.concat(comp_hets)
    comphet_df.name = 'comp_het'
else:
    print 'No compound het variants found.'
    comphet_df = pd.DataFrame()
    comphet_df.name = 'comp_het'

## Results

In [ ]:
# report variant counts
def report_result_counts(results_df):
    if len(results_df) > 0:
        print str(len(results_df)) + ' {} variants found. \n'.format(results_df.name)
        phenotype = results_df['phenotype'].value_counts()
        genes = results_df['gene'].unique()
        print 'Phenotype: \n', phenotype, '\n'
        print 'Affected gene(s): \n', genes, '\n'
    else:
         print "No {} variants found. \n".format(results_df.name)
        
print "Variants found:"
report_result_counts(nb_dominant)
report_result_counts(nb_hom_recessive)
report_result_counts(comphet_df)
print "\n"

### Saving output

In [ ]:
# add from_parent column to dom and hom_rec so we can keep info for comp_het
nb_dominant['from_parent'] = 'NA'
nb_hom_recessive['from_parent'] = 'NA'

# merge and mark predicted variants
def merge_predictors(df, out_list):
    if len(df) > 0:
        df['var_type'] = df.name
        out_list.append(df)
        print "Saving {} {} variants to current working directory".format(len(df), df.name)
    else:
        print "No {} variants to save.".format(df.name)

# list to store patho output
predict_list = []            

merge_predictors(nb_dominant, predict_list)
merge_predictors(nb_hom_recessive, predict_list)
merge_predictors(comphet_df, predict_list)

# merge results
merged_patho = pd.concat(predict_list, axis=0)

# remove unnecessary columns
merge_df = merged_patho[np.r_[0:41, 43, 44, 46, 49:53, 55, 56]]

# save to file
merge_df.to_csv('predicted_ACMG.csv', header=True, encoding='utf-8', index=False)

### Resources

- dbNSFP: 3.0a  
- SnpEff: 4.1h (build 2015-08-03) with GRCh37.74  
- ClinVar:  October 30, 2015 build 
- Kaviar: 50810-Public 